In [1]:
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
sys.path.append("C:/dev/pycmqlib3/")
sys.path.append("C:/dev/pycmqlib3/misc_scripts/")
warnings.filterwarnings('ignore')
import misc
import data_handler as dh
import plotly_examples
import backtest
import dbaccess
from sqlalchemy import create_engine
%matplotlib inline

ferrous_products_mkts = ['rb', 'hc', 'i', 'j', 'jm']
ferrous_mixed_mkts = ['ru', 'FG', 'ZC', 'SM', "SF"]
base_metal_mkts = ['cu', 'al', 'zn', 'pb', 'ni', 'sn']
precious_metal_mkts = ['au', 'ag']
ind_metal_mkts = ferrous_products_mkts + ferrous_mixed_mkts + base_metal_mkts  
petro_chem_mkts = ['l', 'pp', 'v', 'TA', 'MA', 'bu'] #, 'sc', 'fu', 'eg']
ind_all_mkts = ind_metal_mkts + petro_chem_mkts
ags_oil_mkts = ['m', 'RM', 'y', 'p', 'OI', 'a', 'c', 'cs'] #, 'b']
ags_soft_mkts = ['CF', 'SR', 'jd']#, 'AP', 'sp']
ags_all_mkts = ags_oil_mkts + ags_soft_mkts
eq_fut_mkts = ['IF', 'IH', 'IC']
bond_fut_mkts = ['T', 'TF']
fin_all_mkts = eq_fut_mkts + bond_fut_mkts
commod_all_mkts = ind_all_mkts + ags_all_mkts + precious_metal_mkts
all_markets = commod_all_mkts + fin_all_mkts


In [2]:
field_list = ['open', 'high', 'low', 'close', 'volume', 'openInterest', 'contract', 'shift']

start_date = datetime.date(2014,4,1)
end_date = datetime.date(2020,7,10)

sim_products = ['SR'] #['cu', 'al', 'zn', 'pb', 'ni', 'sn'] # ['ZC', 'jm', 'ru', 'FG'] # 
need_shift = 1
freq = 'd'

args = {'n': 1, 'roll_rule': '-35b', 'freq': freq, 'need_shift': need_shift}
ferrous_products_args = args
ferro_mixed_mkt_args = args
base_args = {'n': 1, 'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift}
ni_args = {'n': 1, 'roll_rule': '-40b', 'freq': freq, 'need_shift': need_shift}
eq_args = {'n': 1, 'roll_rule': '-1b', 'freq': freq, 'need_shift': need_shift}
bond_args = {'n': 1, 'roll_rule': '-20b', 'freq': freq, 'need_shift': need_shift}
precious_args = {'n': 1, 'roll_rule': '-25b', 'freq': freq, 'need_shift': need_shift}

df_list = []
for idx, asset in enumerate(sim_products):
    use_args = args
    if asset in eq_fut_mkts:
        use_args = eq_args
    elif asset in ['cu', 'al', 'zn', 'pb', 'sn']:
        use_args = base_args
    elif asset in ['ni']:
        use_args = ni_args
    elif asset in bond_fut_mkts:
        use_args = bond_args
    elif asset in precious_metal_mkts:
        use_args = precious_args
    use_args['start_date'] = max(backtest.sim_start_dict[asset], start_date)
    use_args['end_date'] = end_date
    print("loading mkt = %s, args = %s" % (asset, use_args))
    df = misc.nearby(asset, **use_args)
    if freq == 'm':
        df = misc.cleanup_mindata(df, asset)
    if (idx == 0) and (freq == 'm'):
        df_list.append(df[['date', 'min_id']])
    print(dh.SMAVAR(df, 63))
    df['STDEV'] = dh.STDEV(df, 63)
    print(df[-20:])
    xdf = df[field_list]    
    xdf.columns = [(asset, col) for col in xdf.columns]
    df_list.append(xdf)



loading mkt = SR, args = {'n': 1, 'roll_rule': '-35b', 'freq': 'd', 'need_shift': 1, 'start_date': datetime.date(2014, 4, 1), 'end_date': datetime.date(2020, 7, 10)}
            MA_CLOSE_63  SVAR_CLOSE_63
date                                  
2014-04-01          NaN            NaN
2014-04-02          NaN            NaN
2014-04-03          NaN            NaN
2014-04-04          NaN            NaN
2014-04-08          NaN            NaN
...                 ...            ...
2020-07-06  5101.746032   16872.348199
2020-07-07  5098.396825   15884.810784
2020-07-08  5097.015873   15338.523558
2020-07-09  5096.285714   15089.759637
2020-07-10  5094.174603   14320.175863

[1530 rows x 2 columns]
              open    high     low   close  volume  openInterest contract  \
date                                                                        
2020-06-11  5102.0  5131.0  5011.0  5042.0  708966        439795    SR009   
2020-06-12  5017.0  5085.0  4984.0  5076.0  564967        421007    SR0